# Setup

In [1]:
!pip install 'crewai[tools]' -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.9/542.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.2 MB/s eta 0:00:00

In [2]:
%cd /content/drive/MyDrive/AI Agents/CrewAI/Job Interview Prep Part 2

/content/drive/MyDrive/AI Agents/CrewAI/Job Interview Prep Part 2


In [3]:
# Retrieve API Keys and store them as Environment variables
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')
serper_api_key = userdata.get('serper_api')

import os
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [4]:
# import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [5]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature  = 0.5)

# User Inputs

In [7]:
# Call for inputs: Interviewer, company, job position, job description:
interviewer = input("Enter the name of the interviewer (if unknown include None): ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

Enter the name of the interviewer (if unknown include None): Matthias Glöel
Enter the name of the company: Beat81
Enter the job position: Data Analytics Working Student
Enter the job description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science,

# Research Crew

## Research AI Agent

In [8]:
# Create the Research AI Agent
research_agent = Agent(
    role = "Research Agent",
    goal = "conduct in-depth research",
    backstory = """ As a Research Specialist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations o the task""",
    llm = llm,
    tools = [ScrapeWebsiteTool(),  SerperDevTool()])

## Research Tasks

In [9]:
# Define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends.
    Focus on recent news and developments from last 12 months""",
    expected_output = f""" A comprehensive report in Markdown with bullet point about {company} and its indutry""",
    agent = research_agent)

In [10]:
# Research Interviewer Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional career",
    expected_output = f"""A comprehensive report in Markdown with bullet point about {interviewer}""",
    agent = research_agent)

## AI Coach + Interview Prep

In [11]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm)

In [12]:
# Prepare the lsit of questions
define_questions_task = Task(
    description = f"""
    Define a list a of questions to prepare the user for the job {job_position} based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f""" a list of 20 questions to ask the user in Markdown format that cover: Culture and Team fit, Job position Fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task])

## Assemble the Crew

In [13]:
# Assembling the crew
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential)
result = crew.kickoff({"topic": "Write a list of question to prepare for the the interview"})

# Agent: Research Agent
## Task: 
    Research about Beat81, their position in the sector and recent industry trends.
    Focus on recent news and developments from last 12 months


# Agent: Research Agent
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Beat81 news 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'Beat81 news 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Winter Warrior Challenge 2023 - BEAT81', 'link': 'https://www.beat81.com/lp/winter-warrior-challenge-2023', 'snippet': '\u200dThe goal of the challenge is to participate in a total of 15 BEAT81 classes between November 20, 2023, and December 31, 2023. ... GRITSpot GmbH is also not ...', 'position': 1}, {'title': 'it was a blast   Thanks to everyone who joined our little BEAT81 ...', 'link': 'https://www.instagram.com/beat81/reel/CrGp6lsORBh/', 'snippet': 'FIBO 2023 - it was a blast Thanks to everyone who joined our little BEAT81 booth at the @fi

## Exporting

In [14]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

Here is a list of 20 questions to prepare for the Data Analytics Working Student position at Beat81, covering aspects of Culture and Team Fit, Job Position Fit, Background and Ways of Working, and Growth Mindset:

### Culture and Team Fit
1. How do you align with Beat81's mission to enhance fitness experiences through technology?
2. Can you describe a time when you contributed to a team culture in a dynamic and fast-paced environment?
3. How do you engage with team rituals and events to build a strong team culture?
4. What interests you about working in the fitness and health industry, particularly with a company like Beat81?
5. How do you handle feedback and adapt to changes in a start-up culture?

### Job Position Fit
6. How do you prioritize and manage multiple data requests from different stakeholders?
7. Can you provide an example of a dashboard you created and how it was used to drive business decisions?
8. Describe your experience with SQL and Python in previous projects.
9. How familiar are you with data visualization tools like Tableau or Lookerstudio?
10. What experience do you have with data pipelines, and how have you contributed to their improvement or scaling?

### Background and Ways of Working
11. How does your academic background in a quantitative field prepare you for this role?
12. Describe a project where you collaborated with multiple teams (e.g., Tech, Product, Marketing) to achieve a common goal.
13. How do you ensure accuracy and reliability in your data analysis?
14. What is your approach to learning new technologies or tools that are required for a project?
15. How do you balance working independently on stand-alone projects with collaborating in a team environment?

### Growth Mindset
16. How do you stay updated with the latest trends in data analytics and technology?
17. Can you share an experience where you turned a failure or challenge into a learning opportunity?
18. How do you set personal and professional development goals for yourself in a role like this?
19. What motivates you to continuously improve your skills in data analytics?
20. How do you envision contributing to Beat81's growth and innovation in the fitness industry?

These questions are designed to help you reflect on your experiences and aspirations, ensuring you are well-prepared for an interview at Beat81.

In [15]:
# Export each of the tasks output
# Exporting the  company report
file_name = f"{company}_report.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[0].raw)

# Export the interviewer report
file_name = f"{interviewer}_report.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[1].raw)

# Export the interview questions
file_name = f"{job_position}_questions.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[2].raw)

# Interview Prep

In [16]:
# Import the questions from the directory
with open(f"{job_position}_questions.txt", "r") as file:
    questions = file.read()

In [18]:
# AI Agent that tasks a question
interviewer_agent = Agent(
    role = "Interviewer Agent",
    goal = f"Ask all question from {questions} to the user",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm)

In [22]:
# Interview Task
interview_prep_task = Task(
    description = f"Ask a question from {questions} to the user",
    expected_output = f"Ask only one question at a time",
    agent = interviewer_agent,
    human_input = True)

In [24]:
# Feedback Task
feedback_task = Task(
    description = "Give feedback to the user on its answer",
    expected_output=" Markdown answer of what was good, could be improved and how to take it to the next level",
    agent = coach_agent,
    context = [interview_prep_task],
    human_input = True
    )

## Interview Prep Crew

In [25]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.sequential)
result = crew.kickoff({"topic": f"Ask me all questions from {questions}"})

# Agent: Interviewer Agent
## Task: Ask a question from Here is a list of 20 questions to prepare for the Data Analytics Working Student position at Beat81, covering aspects of Culture and Team Fit, Job Position Fit, Background and Ways of Working, and Growth Mindset:

### Culture and Team Fit
1. How do you align with Beat81's mission to enhance fitness experiences through technology?
2. Can you describe a time when you contributed to a team culture in a dynamic and fast-paced environment?
3. How do you engage with team rituals and events to build a strong team culture?
4. What interests you about working in the fitness and health industry, particularly with a company like Beat81?
5. How do you handle feedback and adapt to changes in a start-up culture?

### Job Position Fit
6. How do you prioritize and manage multiple data requests from different stakeholders?
7. Can you provide an example of a dashboard you created and how it was used to drive business decisions?
8. Describe your exp

In [27]:
# Assemble the interview Prep Crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.hierarchical,
    manager_llm = ChatOpenAI(model_name = "gpt-4o", temperature  = 0),
    memory = True,
    planning = True)
result = crew.kickoff({"topic": f"Ask me all questions from {questions}"})

 
[2025-02-10 11:29:25][INFO]: Planning the crew execution
# Agent: Crew Manager
## Task: Ask a question from Here is a list of 20 questions to prepare for the Data Analytics Working Student position at Beat81, covering aspects of Culture and Team Fit, Job Position Fit, Background and Ways of Working, and Growth Mindset:

### Culture and Team Fit
1. How do you align with Beat81's mission to enhance fitness experiences through technology?
2. Can you describe a time when you contributed to a team culture in a dynamic and fast-paced environment?
3. How do you engage with team rituals and events to build a strong team culture?
4. What interests you about working in the fitness and health industry, particularly with a company like Beat81?
5. How do you handle feedback and adapt to changes in a start-up culture?

### Job Position Fit
6. How do you prioritize and manage multiple data requests from different stakeholders?
7. Can you provide an example of a dashboard you created and how it was 